In [45]:
from IPython.core.display import display
from sympy import pdiv, div, poly, degree, Poly
from sympy.abc import x,y
from sympy import ZZ, GF
from sympy.solvers import solve
from sympy import Symbol
from sympy.solvers.diophantine import diophantine
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt
from sympy.matrices.dense import casoratian
from sympy.matrices.matrices import _iszero, _find_reasonable_pivot, _simplify
from  sympy import gcd

In [2]:
# start=poly(x**5+2*x**4-2*x**3+9*x**2-35*x-5)
# dom = 2

In [3]:
p = 7
m = poly(x**4+3*x-2, domain=GF(p, symmetric=False))
g = poly(2*x**3+3*x**2-4*x+5, domain=GF(p, symmetric=False))

In [4]:
del g


In [5]:
# p = 7
m = poly(x**4+3*x-2, domain=GF(p, symmetric=False))


In [6]:
arr = []
for i in range(0, m.degree() ):
    h = poly(x**(p*i),x, domain=GF(p, symmetric=False))
    r = h % m
    #print(r)
    new_arr = r.all_coeffs()
    arr.append([0]*(m.degree()-len(new_arr)))
    arr[i].extend(new_arr)
#print(arr)
arr=Matrix(arr[::-1]).transpose()-eye(4)
#print(arr)
arr=list(arr)
for i in range(0, len(arr)):
        arr[i] = poly(arr[i],x, domain=GF(p, symmetric=False))
print(arr)
arr = Matrix(4,4, arr)
arr
#list(arr)

[Poly(6, x, modulus=7), Poly(6, x, modulus=7), Poly(2, x, modulus=7), Poly(0, x, modulus=7), Poly(1, x, modulus=7), Poly(4, x, modulus=7), Poly(0, x, modulus=7), Poly(0, x, modulus=7), Poly(6, x, modulus=7), Poly(1, x, modulus=7), Poly(1, x, modulus=7), Poly(0, x, modulus=7), Poly(3, x, modulus=7), Poly(3, x, modulus=7), Poly(1, x, modulus=7), Poly(0, x, modulus=7)]


Matrix([
[Poly(6, x, modulus=7), Poly(6, x, modulus=7), Poly(2, x, modulus=7), Poly(0, x, modulus=7)],
[Poly(1, x, modulus=7), Poly(4, x, modulus=7), Poly(0, x, modulus=7), Poly(0, x, modulus=7)],
[Poly(6, x, modulus=7), Poly(1, x, modulus=7), Poly(1, x, modulus=7), Poly(0, x, modulus=7)],
[Poly(3, x, modulus=7), Poly(3, x, modulus=7), Poly(1, x, modulus=7), Poly(0, x, modulus=7)]])

In [22]:
#алгоритм нахождения ядра
rows, cols = arr.rows, arr.cols
mat = list(arr)
iszerofunc = _iszero
simpfunc = _simplify
zero_above = True
def get_col(i):
    return mat[i::cols]

def row_swap(i, j):
    mat[i*cols:(i + 1)*cols], mat[j*cols:(j + 1)*cols] = \
        mat[j*cols:(j + 1)*cols], mat[i*cols:(i + 1)*cols]

def cross_cancel(a, i, b, j):
    """Does the row op row[i] = a*row[i] - b*row[j]"""
    q = (j - i)*cols
    for p in range(i*cols, (i + 1)*cols):
        mat[p] = a*mat[p] - b*mat[p + q]
piv_row, piv_col = 0, 0
pivot_cols = []
swaps = []
while piv_col < cols and piv_row < rows:
    pivot_offset, pivot_val, \
    assumed_nonzero, newly_determined = _find_reasonable_pivot(
        get_col(piv_col)[piv_row:], iszerofunc, simpfunc)

    # _find_reasonable_pivot may have simplified some things
    # in the process.  Let's not let them go to waste
    for (offset, val) in newly_determined:
        offset += piv_row
        mat[offset*cols + piv_col] = val

    if pivot_offset is None:
        piv_col += 1
        continue

    pivot_cols.append(piv_col)
    if pivot_offset != 0:
        row_swap(piv_row, pivot_offset + piv_row)
        swaps.append((piv_row, pivot_offset + piv_row))

    # if we aren't normalizing last, we normalize
    # before we zero the other rows
   

    # zero above and below the pivot
    for row in range(rows):
        # don't zero our current row
        if row == piv_row:
            continue
        # don't zero above the pivot unless we're told.
        if zero_above is False and row < piv_row:
            continue
        # if we're already a zero, don't do anything
        val = mat[row*cols + piv_col]
        if iszerofunc(val):
            continue

        cross_cancel(pivot_val, row, val, piv_row)
    piv_row += 1
#poly(1,x, domain=GF(p, symmetric=False))
#if normalize_last is True and normalize is True:
for piv_i, piv_j in enumerate(pivot_cols):
    pivot_val = mat[piv_i*cols + piv_j]
    mat[piv_i*cols + piv_j] = poly(1,x, domain=GF(p, symmetric=False))
    for p in range(piv_i*cols + piv_j + 1, (piv_i + 1)*cols):
        mat[p] = div(mat[p], pivot_val)[0]
print(mat)
q=Matrix(rows, cols, mat)
pivots= tuple(pivot_cols)
reduced = q
free_vars = [i for i in range(q.cols) if i not in pivots]

basis = []
for free_var in free_vars:
    # for each free variable, we will set it to 1 and all others
    # to 0.  Then, we will use back substitution to solve the system
    vec = [poly(0,x, domain=GF(p, symmetric=False))]*q.cols
    vec[free_var] = poly(1,x, domain=GF(p, symmetric=False))
    for piv_row, piv_col in enumerate(pivots):
        vec[piv_col] -= reduced[piv_row, free_var]
    basis.append(vec)
kernel = [Matrix(q.cols, 1, b) for b in basis]
kernel

[Poly(1, x, modulus=7), Poly(0, x, modulus=7), Poly(2, x, modulus=7), Poly(0, x, modulus=7), Poly(0, x, modulus=7), Poly(1, x, modulus=3), Poly(3, x, modulus=7), Poly(0, x, modulus=7), Poly(0, x, modulus=7), Poly(0, x, modulus=7), Poly(0, x, modulus=7), Poly(0, x, modulus=7), Poly(0, x, modulus=7), Poly(0, x, modulus=7), Poly(0, x, modulus=7), Poly(0, x, modulus=7)]


[Matrix([
 [Poly(5, x, modulus=7)],
 [Poly(4, x, modulus=7)],
 [Poly(1, x, modulus=7)],
 [Poly(0, x, modulus=7)]]),
 Matrix([
 [Poly(0, x, modulus=7)],
 [Poly(0, x, modulus=7)],
 [Poly(0, x, modulus=7)],
 [Poly(1, x, modulus=7)]])]

In [38]:
numeric_coef = list()
for i in kernel[0]:
    numeric_coef.append(i.LC())


In [51]:
t = Poly(numeric_coef, x, domain = GF(7))
display(m)
display(t - 0)
for i in range(0, 8):
    display(gcd(t - Poly(i, x,domain = GF(7)), m))

Poly(x**4 + 3*x + 5, x, modulus=7)

Poly(-2*x**3 - 3*x**2 + x, x, domain='ZZ')

Poly(1, x, modulus=7)

Poly(1, x, modulus=7)

Poly(1, x, modulus=7)

Poly(x**2 + 6*x + 2, x, modulus=7)

Poly(1, x, modulus=7)

Poly(1, x, modulus=7)

Poly(x**2 + x + 6, x, modulus=7)

Poly(1, x, modulus=7)

(Poly(-3, x, modulus=7), Poly(0, x, modulus=7))

In [179]:
c=q.nullspace()[0]
#c=Matrix([1,-2,-4,0])
c

Matrix([
[-2],
[-3],
[ 1],
[ 0]])

In [180]:
a*c

Matrix([
[  7],
[-14],
[  0],
[  0]])